# Recommendation System of books from Amazon

## Libraries used:
- langchain, chromadb, openai, csv loader, opeanai embeddings, retrievalQA

In [13]:
# import libraries

from langchain.document_loaders import CSVLoader
from langchain.vectorstores import Chroma
import chromadb
from langchain.embeddings import OpenAIEmbeddings
import openai
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain.chat_models import ChatOpenAI

In [14]:
load_dotenv()

True

In [15]:
# Carregando o csv no langchain

csv_loader = CSVLoader('livros-skoob.csv')
csv_file = csv_loader.load()

In [16]:
# Dividindo o texto em pedaços

csv_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

csv_document = csv_splitter.split_documents(csv_file)

len(csv_document)


7907

In [17]:
# find a specific text in the csv file

csv_document[1]

Document(page_content='Unnamed: 0: 1\nTitle: Black Holes (L) : The Reith Lectures [Paperback] Hawking, Stephen\nAuthor: Stephen Hawking\nMain Genre: Arts, Film & Photography\nSub Genre: Cinema & Broadcast\nType: Paperback\nPrice: ₹99.00\nRating: 4.5', metadata={'source': 'amazon_books_cleaned.csv', 'row': 1})

In [20]:
# criando vetores com langchain chroma

# 1) persistir o diretório

persist_directory = 'chroma_db'

# # 2) criando os embeddings

embeddings = OpenAIEmbeddings(api_key="")

# # 3) criando os vetores

vector_db = Chroma.from_documents(documents=csv_document, embedding=embeddings, persist_directory=persist_directory)

In [21]:
# persistindo os vetores no disco

vector_db.persist()
vector_db = None

In [43]:
# Carregando o armazenamento de vetores do disco para usar

vector_db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [50]:
# criando o recuperador

retriever = vector_db.as_retriever()


In [51]:
# Verificando o tipo de pesquisa com o recuperador

retriever.search_type

'similarity'

In [52]:
# criando o modelo de prompt

prompt_template = """Você é um sistema de recomendação de livros do Skoob. Use as seguintes partes do contexto para 
responder à pergunta e recomendar um livro ao usuário. Se você não souber a resposta, basta dizer "não sei"".

{context}

Questão: {question}
"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])

In [53]:
# Crie a Chain para o chat com o retriever e o modelo de prompt

qachain = RetrievalQA.from_chain_type(llm=ChatOpenAI(
                                    model_name = "gpt-3.5-turbo",
                                    temperature=0.1,
                                    verbose=False), 
                                    chain_type='stuff',
                                    chain_type_kwargs={'prompt': PROMPT},
                                    retriever=retriever)

In [55]:
doc = qachain('Pode me recomendar um livro sobre inteligência artificial?')
import pprint
pprint.pprint(doc['result'])

('Com base nas informações fornecidas, eu recomendaria o livro "Artificial '
 'Intelligence: A Modern Approach, 4e" de Russell/Norvig. Ele está disponível '
 'em formato de Paperback, com um preço de ₹1,116.01 e possui uma avaliação de '
 '4.3. Espero que goste da recomendação!')


In [57]:
# inititalize the chatbot question and answering with PROMPT 

QUERY = "Algum livro para recomendar do genêro de Fantasia sobre dragões?"

doc_prompt = qachain({"query": QUERY})

import pprint

pprint.pprint(doc_prompt['result'])

('Recomendo o livro "The Serpent and the Wings of Night (Crowns of Nyaxia Book '
 '1)" de Carissa Broadbent. É um livro de fantasia que pode te interessar, '
 'pois envolve dragões em sua história.')
